# Example Notebook for the Psypypeline


## Import the module and setup a pipeline

In [1]:
from psypypeline.psypypeline import Pipeline
pipeline = Pipeline(name="TestPipeline", root="example")

C:\Users\hulin\anaconda3\lib\site-packages\bids\layout\models.py:148: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  warnings.warn("The 'extension' entity currently excludes the leading dot ('.'). "


This loads everything as specified in pipeline.json, which must lie in *<root\>/derivatives/<name\>*
In pipeline.json, one can specify processes (their name, the script in which they are stored and a version of their name which will be used for filenames) and masks (their name and the nii(.gz) file in which they are stored).
the above code loads all of this into memory.
You can call `pipeline.processes` or `pipeline.masks` and compare the output to the content of the *pipeline.json* or the folder (*masks*) or python script (*processes.py** where they *re stored.

In [2]:
pipeline.masks

{'csf': nltools.data.brain_data.Brain_Data(data=(238955,), Y=0, X=(0, 0), mask=MNI152_T1_2mm_brain_mask.nii.gz, output_file=[]),
 'rois': nltools.data.brain_data.Brain_Data(data=(238955,), Y=0, X=(0, 0), mask=MNI152_T1_2mm_brain_mask.nii.gz, output_file=[])}

In [3]:
pipeline.processes

{'denoise': <psypypeline.psypypeline.Process at 0x1f009094af0>,
 'smooth': <psypypeline.psypypeline.Process at 0x1f009094100>,
 'EVcentrality': <psypypeline.psypypeline.Process at 0x1f009081370>}

Here we see a new class: `Processes`. Calling the `__dict__` of one of the processes, tells us more about their content:

In [4]:
pipeline.processes["denoise"].__dict__

{'key': 'denoised',
 'process': <function processes.denoise(pipeline, sub, data, global_spike_cutoff=3, diff_spike_cutoff=3)>,
 'readable': <function psypypeline.psypypeline.Process.__init__.<locals>.<lambda>(args)>}

## Load data using the pipeline

Now, loading the data is easy:

In [5]:
pipeline.load_data(sub="S01")

...loading the unprocessed file of subject S01


nltools.data.brain_data.Brain_Data(data=(16, 238955), Y=0, X=(0, 0), mask=MNI152_T1_2mm_brain_mask.nii.gz, output_file=[])

In [6]:
pipeline.load_data(sub="S01", smooth={})

...found sub-S01_smoothed_bold.nii.gz


nltools.data.brain_data.Brain_Data(data=(16, 238955), Y=0, X=(0, 0), mask=MNI152_T1_2mm_brain_mask.nii.gz, output_file=[])

As we can see here, just supplying the name of a subject loads the unprocesses file. Additionally supplying keywords like `smooth` applies processes from `pipeline.processes` to them, in the order of appearance. As you can see, no key is supplied to the keyword, which means that the process will run with the default parameters. We can specify the parameters by supplying *<key\>-<value\>* pairs in the dictionary. But how do we know which parameters are allowed (apart from checking our own code again)?


In [7]:
pipeline.processes["smooth"].process

<function processes.smooth(pipeline, sub, data, fwhm=6)>

So smoothing data with a different kernel looks like this: 

In [8]:
pipeline.load_data("S01", smooth={"fwhm": 3})

...found sub-S01_smoothed-{fwhm-3}_bold.nii.gz


nltools.data.brain_data.Brain_Data(data=(16, 238955), Y=0, X=(0, 0), mask=MNI152_T1_2mm_brain_mask.nii.gz, output_file=[])

Note that, if you run of the `load_data` cells multiple times, it speeds up considerably and there is less output. That is because, if not specified otherwise, the loading process first checks if already applied this process and stored it. You can change that an other behavior of the function. Look at the docstring to find out more.

In [9]:
?pipeline.load_data

Signature:
pipeline.load_data(
    sub,
    return_type='Brain_Data',
    write='all',
    force='none',
    verbose=True,
    reload=True,
    **processes,
) -> nltools.data.brain_data.Brain_Data
Docstring:
Load data from pipeline.root/derivatives/pipeline.name and/or applies processes from
pipeline.processes to it.
By default, first checks wether the processes have been applied and saved before and 
then loads them. By default, saves all the intermediate steps

Parameters
----------
sub : str
    Name of the subject to load the process from.
return_type : str, optional
    Type the return value. Must be one of "path", "Brain_Data". If "path" and write="none" and file does not exist,
    throws an Error, as path does not exist. By default "Brain_Data"
write : str, optional
    Wether to save the intermediate and the last step when applying processes. Must be one of "none" (no step is saved),
    "main" (only endresult is saved) or "all" (all intermediate steps are saved). By default "